In [1]:
import sys
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.append(r'reflutils')
from reflutils import FileDialog, Refl

In [2]:
xrrDataBase = FileDialog.getDirectory(title="Select the XRR Database")

In [3]:
SampleName = xrrDataBase.name

In [9]:
energies = [e.name for e in xrrDataBase.iterdir() if e.is_dir()]
e = widgets.Dropdown(options = energies, description = "Energy")
e

Dropdown(description='Energy', options=('250.0', '275.0', '280.0', '281.0', '281.2', '281.4', '281.6', '281.8'…